In [53]:
%matplotlib inline
import pandas as pd
import datetime
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch.utils.data.dataloader import default_collate
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from IPython.display import clear_output
from torch.autograd import Variable

In [31]:
class IMUDataset(Dataset):
    def __init__(self, mode = 'test', transform = None):
        if mode == 'train' :
            self.df = pd.read_csv('data_preparation/train.csv', header = 0, index_col = 0)
        elif mode == 'test' :
            self.df = pd.read_csv('data_preparation/test.csv', header = 0, index_col = 0)
        elif mode == 'val' :
            self.df = pd.read_csv('data_preparation/val.csv', header = 0, index_col = 0)
        self.transform = transform
        self.df = self.df.reset_index()
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        y = self.df.iloc[idx : idx + 80, 3 : ].values
        ind = np.argmax(np.sum(y, axis = 0))
        label = np.zeros_like(self.df.iloc[0, 3 : ].values)
        label = label.astype(float)
        label[ind] = 1
        x = self.df.iloc[idx : idx + 80, : 3].values
        x = x.astype(float)
        assert(x.shape == (80, 3))
        assert(label.shape == (6, ))
        return x, label
        
trainset = IMUDataset(mode = 'train')
valset = IMUDataset(mode = 'val')
testset = IMUDataset(mode = 'test')

In [34]:
batch_size = 64
def my_collate(batch):
    "Puts each data field into a tensor with outer dimension batch size"
    batch = list(filter(lambda x : x is not None, batch))
    return default_collate(batch)

train_indices = [(i * 80) for i in range(len(trainset) // 80)]
val_indices = [(i * 80) for i in range(len(valset) // 80)]
test_indices = [(i * 80) for i in range(len(testset) // 80)]

trainloader = DataLoader(trainset, batch_size = batch_size, sampler = SubsetRandomSampler(train_indices), collate_fn = my_collate, drop_last = True)
valloader = DataLoader(valset, batch_size = batch_size, sampler = SubsetRandomSampler(val_indices), collate_fn = my_collate, drop_last = True)
testloader = DataLoader(testset, batch_size = batch_size, sampler = SubsetRandomSampler(test_indices), collate_fn = my_collate, drop_last = True)

# for idx, (x, y) in enumerate(trainloader) :
#     print(x.shape, y.shape)

In [41]:
class ConvAutoEncoder(nn.Module):
    def __init__(self, code_size):
        super(ConvAutoEncoder, self).__init__()
        self.code_size = code_size
        # defining layers
        # input size - batch_size x 3 x 100
        conv1 = nn.Conv1d(3, 10, 3)
        # after conv1 - batch_size x 10 x 98
        conv2 = nn.Conv1d(10, 20, 1)
        # after conv2 - batch_size x 10 x 98
        enc_fc = nn.Linear(10 * 98, self.code_size)
        
        dec_fc = nn.Linear(self.code_size, 10 * 98)
        deconv2 = nn.ConvTranspose1d(20, 10, 1)
        deconv1 = nn.ConvTranspose1d(10, 3, 3)
        
    def forward(self, signal):
        code = self.encode(signal)
        out = self.decode(code)
        return out, code
    
    def encode(signal):
        signal = signal.view(16, 3, 80)
        code = F.sigmoid(self.conv1(signal))
        code = F.sigmoid(self.conv2(code))
        code = F.sigmoid(self.enc_fc(code))
        return code
    
    def decode(code):
        out = F.sigmoid(self.dec_fc(code))
        out = F.sigmoid(self.deconv2(out))
        out = F.sigmoid(self.deconv1(out))
        return out
    
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # defining layers
        self.conv1 = nn.Conv1d(3, 5, 3)
        self.conv2 = nn.Conv1d(5, 10, 5)
        self.conv3 = nn.Conv1d(10, 10, 5)
        self.fc = nn.Linear(70 * 10, 6)
        
        nn.init.xavier_uniform_(self.conv1.weight, gain = nn.init.calculate_gain('relu'))
        nn.init.xavier_uniform_(self.conv2.weight, gain = nn.init.calculate_gain('relu'))
        nn.init.xavier_uniform_(self.conv3.weight, gain = nn.init.calculate_gain('relu'))
        nn.init.xavier_uniform_(self.conv3.weight, gain = nn.init.calculate_gain('sigmoid'))
        
    def forward(self, signal):
        signal = signal.view(batch_size, 3, 80)
        out = F.relu(self.conv1(signal))
        out = F.relu(self.conv2(out))
        out = F.relu(self.conv3(out))
        out = out.view(-1, 70 * 10)
        out = F.log_softmax(self.fc(out), dim = 1)
        return out
    
Net = ConvNet()

In [14]:
def output_size(n, f, p = 0, s = 1):
    return (((n + 2 * p - f) / s) + 1)

print(output_size(80, 3))
print(output_size(78, 5))
print(output_size(74, 5))

78.0
74.0
70.0


In [44]:
import torch.optim as optim

criterion = nn.NLLLoss()
optimizer = optim.Adam(Net.parameters(), lr=0.001)